In [ ]:
from pathlib import Path
import numpy as np
import numpy.typing as npt
import h5py
import matplotlib.pyplot as plt
from topostats.io import LoadScans
from topostats.plotting import Colormap
from scipy.ndimage import binary_dilation

colormap = Colormap()
cmap = colormap.get_cmap()
vmin = -3
vmax = 4

from fig_colours_and_functions import (
    good_pixels_colormap,
    good_pixels_alpha,
    good_pixels_colour,
    good_pixels_colour_alpha,
    regular_pixels_colormap,
    regular_pixels_alpha,
    branch_underlying_colour,
    branch_overlying_colour,
    good_mask_color,
    good_mask_alpha,
    molecule_1_colour,
    molecule_2_colour,
    overlay_mask,
    dilate_branches_in_mask,
    plot_overlay_dilated_branches_in_mask,
    overlay_mask_with_specific_colour,
    plot_overlay_dilated_branches_in_mask_specific_colours,
)

In [ ]:
base_dir = Path("/Users/sylvi/topo_data/topostats_2/datasets/20250528_RA_dose_0GY_picoz")
data_dir = base_dir / "output_fig_nodestats"
figure_dir = Path("/Users/sylvi/topo_data/topostats_2/figures/fig-pipeline")
assert figure_dir.exists()
assert data_dir.exists()

image_filename = "20250528_picoz_0RA_tip_0.0_00029"
image_topostats_file_path = data_dir / "processed" / (f"{image_filename}.topostats")
assert image_topostats_file_path.exists()
loadscans = LoadScans(img_paths=[image_topostats_file_path], channel="dummy")
loadscans.get_data()
loadscans_dict = loadscans.img_dict
image_dict = loadscans_dict[image_filename]
print(image_dict.keys())
image = image_dict["image"]
p2nm = image_dict["pixel_to_nm_scaling"]
print(f"pixel to nm scaling: {p2nm} nm/pixel")
print(f"image size: {image.shape} px, {image.shape[0]*p2nm:.2f} x {image.shape[1]*p2nm:.2f} nm")
image_disordered_traces = image_dict["disordered_traces"]["above"]
image_nodestats = image_dict["nodestats"]["above"]
image_ordered_tracing = image_dict["ordered_traces"]["above"]

image_nodestats_images_grain1 = image_nodestats["images"]["grain_0"]
image_nodestats_images_grain1_grain = image_nodestats_images_grain1["grain"]
image_nodestats_images_grain1_grain_image = image_nodestats_images_grain1_grain["grain_image"]

image_disordered_traces_grain1 = image_disordered_traces["grain_0"]
image_disordered_traces_grain1_branch_indexes = image_disordered_traces_grain1["branch_indexes"]

fig, ax = plt.subplots(figsize=(10, 10))
ax.imshow(image_nodestats_images_grain1_grain_image, cmap=cmap, vmin=vmin, vmax=vmax)
ordered_skeleton_mask = image_disordered_traces_grain1_branch_indexes.astype(bool)
# plot_overlay_dilated_branches_in_mask(ordered_skeleton_mask, colourmap=good_pixels_colormap, alpha=good_pixels_alpha)
plot_overlay_dilated_branches_in_mask_specific_colours(
    ordered_skeleton_mask, colours={1: good_pixels_colour}, alpha=good_pixels_colour_alpha
)

for node_id, node_image_data in image_nodestats_images_grain1["nodes"].items():
    print(f"node_id: {node_id}")
    node_branch_mask = node_image_data["node_branch_mask"]
    node_area_skeleton = node_image_data["node_area_skeleton"]
    node_avg_mask = node_image_data["node_avg_mask"]

    # plt.imshow(node_branch_mask)
    # plt.title(f"node_id: {node_id} branch mask")
    # plt.show()
    # plt.imshow(node_area_skeleton)
    # plt.title(f"node_id: {node_id} area skeleton")
    # plt.show()
    # plt.imshow(node_avg_mask)
    # plt.title(f"node_id: {node_id} avg mask")
    # plt.show()

    # make masked array for node branch mask
    print(np.unique(node_branch_mask))
    plot_overlay_dilated_branches_in_mask_specific_colours(
        node_branch_mask, colours={1: branch_underlying_colour, 2: branch_overlying_colour}, alpha=1
    )
plt.savefig(figure_dir / f"fig-ordered-tracing-picoz-branch-crossings", bbox_inches="tight")
plt.show()

fig, ax = plt.subplots(figsize=(10, 10))
picoz_spline_g1_m0_coords = image_dict["splining"]["above"]["grain_0"]["mol_0"]["spline_coords"]
plt.imshow(image_nodestats_images_grain1_grain_image, cmap=cmap, vmin=vmin, vmax=vmax)
plt.plot(
    picoz_spline_g1_m0_coords[:, 1],
    picoz_spline_g1_m0_coords[:, 0],
    color=molecule_1_colour,
    linewidth=4,
)
plt.savefig(figure_dir / f"fig-ordered-tracing-picoz-mol-0-splined", bbox_inches="tight")
plt.show()

In [ ]:
pipeline_grain_image = image_nodestats_images_grain1_grain_image.copy()


%matplotlib widget

# render the heightmap image as a 3d plot
fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(111, projection="3d")
X = np.arange(pipeline_grain_image.shape[1]) * p2nm  # x coordinates in nm
Y = np.arange(pipeline_grain_image.shape[0]) * p2nm  # y coordinates in nm
X, Y = np.meshgrid(X, Y)
ax.plot_surface(X, Y, pipeline_grain_image, cmap=cmap, vmin=vmin, vmax=vmax)
# set the z limits
ax.set_zlim(0, np.max(pipeline_grain_image)*10)
plt.show()